In [1]:
import csv
import os
import nltk
import random
from nltk import word_tokenize
os.chdir('H:/GEMINI/Results/WATSON/')

In [2]:
f = open('nlp.data.csv', newline = "")
full = csv.reader(f)

In [63]:
full = list(full)
report = []
contrast = []
for row in full:
    report.append(word_tokenize(row[1]))
    contrast.append(row[2])
report = report[1:]
contrast = contrast[1:]

In [94]:
document = [(list(report[i]), contrast[i])
           for i in range(298)]
random.shuffle(document)

In [95]:
len(document)

298

In [113]:
from nltk.corpus import PlaintextCorpusReader
corpus_root = 'H:/GEMINI/Results/WATSON/corpus'
ct_reports = PlaintextCorpusReader(corpus_root, '.*')


In [97]:
all_words = nltk.FreqDist(w.lower() for w in ct_reports.words() if w.isalpha())
word_features = list(all_words)


In [98]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [99]:
featuresets = [(document_features(d), c) for (d,c) in document]
train_set, test_set = featuresets[100:], featuresets[:100]
test = [(a) for (a, b) in featuresets]
classifier = nltk.NaiveBayesClassifier.train(train_set)
classifier.show_most_informative_features(20)

Most Informative Features
        contains(anemia) = True                n : y      =     10.4 : 1.0
contains(administration) = True                y : n      =      9.1 : 1.0
    contains(unenhanced) = True                n : y      =      9.0 : 1.0
          contains(dose) = True                n : y      =      9.0 : 1.0
          contains(lack) = True                n : y      =      7.9 : 1.0
         contains(thick) = True                n : y      =      7.6 : 1.0
       contains(without) = True                n : y      =      7.2 : 1.0
      contains(material) = True                n : y      =      7.1 : 1.0
         contains(renal) = True                n : y      =      7.0 : 1.0
     contains(following) = True                y : n      =      6.8 : 1.0
 contains(opacification) = True                y : n      =      6.4 : 1.0
       contains(smoking) = True                n : y      =      6.2 : 1.0
         contains(blood) = True                n : y      =      6.2 : 1.0

In [100]:
print(nltk.classify.accuracy(classifier, test_set))

0.77


In [101]:
#classifier.classify_many(test)

In [102]:
classifier2 = nltk.classify.DecisionTreeClassifier.train(train_set, entropy_cutoff = 0.5)

In [103]:
print(nltk.classify.accuracy(classifier2, test_set))

0.8


In [104]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

In [105]:
classif = SklearnClassifier(RandomForestClassifier())
classifier3 = classif.train(train_set)

In [106]:
print(nltk.classify.accuracy(classifier3, test_set))

0.77


In [115]:
# try different classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

In [116]:
classifiers = [
    SVC(kernel="linear", C=0.02),
    DecisionTreeClassifier(max_depth=6),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=0.1),
    AdaBoostClassifier()
]
for cl in classifiers:
    classif = SklearnClassifier(cl)
    classifier_i = classif.train(train_set)
    print(cl, end = ': ')
    print(nltk.classify.accuracy(classifier_i, test_set))

SVC(C=0.02, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False): 0.81
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'): 0.83
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=1, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False): 0.75
MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=

In [110]:
# parameter tuning for ada
learning_rates = [0.05, 0.1, 0.5, 1, 1.5, 2]
for learning_rate in learning_rates:
    classif = SklearnClassifier(AdaBoostClassifier(n_estimators = 100, learning_rate = learning_rate))
    classifier_i = classif.train(train_set)
    print(learning_rate, end = ': ')
    print(nltk.classify.accuracy(classifier_i, test_set))

0.05: 0.81
0.1: 0.82
0.5: 0.79
1: 0.86
1.5: 0.79
2: 0.5
